In [1]:
!pip install textsearch
!pip install contractions
import nltk
nltk.download('punkt')
nltk.download('stopwords')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 6.3 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
import pandas as pd

df = pd.read_csv('tmdb_5000_movies.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [5]:
df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [6]:
# Load data and select required columns
df = df[['title', 'tagline', 'overview', 'popularity']]

# Fill missing values in 'tagline' with empty strings
df['tagline'].fillna('', inplace=True)

# Combine 'tagline' and 'overview' into 'description'
df['description'] = df['tagline'].map(str) + ' ' + df['overview']

# Drop rows with missing 'overview' (or 'description')
df.dropna(subset=['description'], inplace=True)

# Sort dataframe by popularity
df = df.sort_values(by=['popularity'], ascending=False)

<ipython-input-6-015c4faef65e>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['tagline'].fillna('', inplace=True)
<ipython-input-6-015c4faef65e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tagline'].fillna('', inplace=True)
<ipython-input-6-015c4faef65e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [7]:
df.head()

,title,tagline,overview,popularity,description
546,Minions,"Before Gru, they had a history of bad bosses","Minions Stuart, Kevin and Bob are recruited by...",875.581305,"Before Gru, they had a history of bad bosses M..."
95,Interstellar,Mankind was born on Earth. It was never meant ...,Interstellar chronicles the adventures of a gr...,724.247784,Mankind was born on Earth. It was never meant ...
788,Deadpool,Witness the beginning of a happy ending,Deadpool tells the origin story of former Spec...,514.569956,Witness the beginning of a happy ending Deadpo...
94,Guardians of the Galaxy,All heroes start somewhere.,"Light years from Earth, 26 years after being a...",481.098624,All heroes start somewhere. Light years from E...
127,Mad Max: Fury Road,What a Lovely Day.,An apocalyptic story set in the furthest reach...,434.278564,What a Lovely Day. An apocalyptic story set in...


In [8]:
import re
import numpy as np
import contractions

In [9]:
# Normalization function to clean up text
stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # Lowercase and remove special characters and whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I | re.A)
    doc = doc.lower()
    doc = doc.strip()
    doc = contractions.fix(doc)
    # Tokenize document
    tokens = nltk.word_tokenize(doc)
    # Remove stopwords
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # Recreate document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

# Using list comprehension to normalize the corpus
norm_corpus = [normalize_document(doc) for doc in df['description']]

# Check the length of normalized corpus
len(norm_corpus)

4800

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
tfidf_matrix = tf.fit_transform(norm_corpus)

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
doc_sim = cosine_similarity(tfidf_matrix)
doc_sim_df = pd.DataFrame(doc_sim)

In [12]:
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,4790,4791,4792,4793,4794,4795,4796,4797,4798,4799
0,1.000000,0.000000,0.000000,0.000000,0.006071,0.008067,0.0,0.000000,0.0,0.000000,...,0.018758,0.000000,0.037930,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.009646
1,0.000000,1.000000,0.000000,0.017839,0.007968,0.000000,0.0,0.012501,0.0,0.014840,...,0.000000,0.000000,0.017564,0.0,0.019152,0.000000,0.000000,0.000000,0.000000,0.007963
2,0.000000,0.000000,1.000000,0.000000,0.017178,0.000000,0.0,0.000000,0.0,0.024326,...,0.000000,0.006903,0.005024,0.0,0.012893,0.000000,0.025975,0.000000,0.027126,0.009340
3,0.000000,0.017839,0.000000,1.000000,0.000000,0.022414,0.0,0.000000,0.0,0.037207,...,0.000000,0.060846,0.025039,0.0,0.036237,0.030516,0.022605,0.000000,0.000000,0.000000
4,0.006071,0.007968,0.017178,0.000000,1.000000,0.004673,0.0,0.064581,0.0,0.000000,...,0.022064,0.019662,0.036561,0.0,0.015826,0.000000,0.076033,0.004516,0.043475,0.011465


In [13]:
movies_list = df['title'].values
movies_list, movies_list.shape

(array(['Minions', 'Interstellar', 'Deadpool', ..., 'Penitentiary',
        'Alien Zone', 'America Is Still the Place'], dtype=object),
 (4800,))

In [14]:
print(movies_list)

['Minions' 'Interstellar' 'Deadpool' ... 'Penitentiary' 'Alien Zone'
 'America Is Still the Place']


In [15]:
movie_idx = np.where(movies_list == 'Avatar')[0][0]
movie_idx

18

In [16]:
movie_similarities = doc_sim_df.iloc[movie_idx].values
movie_similarities

array([0.0104544, 0.       , 0.       , ..., 0.       , 0.       ,
       0.       ])

In [17]:
similar_movie_idxs = np.argsort(-movie_similarities)[1:6]
similar_movie_idxs

array([  76, 3994, 1987, 1263, 1098])

In [18]:
similar_movies = movies_list[similar_movie_idxs]
similar_movies

array(['The Matrix', 'The Inhabited Island', 'Apollo 18',
       'Tears of the Sun', 'Green Zone'], dtype=object)

In [19]:
class MovieRecommenderChatbot:
    def __init__(self):
        # Load and prepare data
        self.df = pd.read_csv('tmdb_5000_movies.csv')
        self.prepare_data()
        self.create_recommendation_system()

    def prepare_data(self):
        # Select required columns
        self.df = self.df[['title', 'tagline', 'overview', 'popularity']]
        self.df['tagline'].fillna('', inplace=True)
        self.df['description'] = self.df['tagline'].map(str) + ' ' + self.df['overview']
        self.df.dropna(subset=['description'], inplace=True)
        self.df = self.df.sort_values(by=['popularity'], ascending=False)

    def normalize_document(self, doc):
        # Lowercase and remove special characters
        doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I | re.A)
        doc = doc.lower()
        doc = doc.strip()
        doc = contractions.fix(doc)

        # Tokenize and remove stopwords
        stop_words = nltk.corpus.stopwords.words('english')
        tokens = nltk.word_tokenize(doc)
        filtered_tokens = [token for token in tokens if token not in stop_words]

        return ' '.join(filtered_tokens)

    def create_recommendation_system(self):
        # Create normalized corpus
        norm_corpus = [self.normalize_document(doc) for doc in self.df['description']]

        # Create TF-IDF matrix
        tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
        self.tfidf_matrix = tf.fit_transform(norm_corpus)

        # Calculate similarity matrix
        self.doc_sim = cosine_similarity(self.tfidf_matrix)
        self.movies_list = self.df['title'].values

    def get_recommendations(self, movie_title):
        try:
            # Find movie index
            movie_idx = np.where(self.movies_list == movie_title)[0][0]

            # Get movie similarities
            movie_similarities = self.doc_sim[movie_idx]

            # Get top 5 similar movies
            similar_movie_idxs = np.argsort(-movie_similarities)[1:6]
            similar_movies = self.movies_list[similar_movie_idxs]

            return similar_movies

        except IndexError:
            return None

    def find_closest_match(self, user_input):
        # Convert titles to lowercase for case-insensitive matching
        user_input = user_input.lower()
        movies_lower = [title.lower() for title in self.movies_list]

        # Find movies that contain the user input
        matches = [title for title in movies_lower if user_input in title]

        if matches:
            # Return the original case version of the first match
            index = movies_lower.index(matches[0])
            return self.movies_list[index]
        return None

    def chat(self):
        print("Movie Recommendation Chatbot")
        print("============================")
        print("Welcome! I can help you find similar movies based on a title.")
        print("Type 'quit' or 'exit' to end the conversation.")

        while True:
            user_input = input("\nEnter a movie title: ").strip()

            if user_input.lower() in ['quit', 'exit']:
                print("Thank you for using the Movie Recommender! Goodbye!")
                break

            if not user_input:
                print("Please enter a movie title.")
                continue

            # Try to find exact or closest match
            movie = self.find_closest_match(user_input)

            if movie is None:
                print(f"Sorry, I couldn't find a movie with title '{user_input}'.")
                continue

            print(f"\nFound movie: {movie}")
            recommendations = self.get_recommendations(movie)

            if recommendations is not None:
                print("\nHere are similar movies you might enjoy:")
                for i, rec in enumerate(recommendations, 1):
                    print(f"{i}. {rec}")
            else:
                print("Sorry, I couldn't generate recommendations for this movie.")

# Create and run the chatbot
if __name__ == "__main__":
    chatbot = MovieRecommenderChatbot()
    chatbot.chat()

<ipython-input-19-5888f42d23e3>:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  self.df['tagline'].fillna('', inplace=True)


Movie Recommendation Chatbot
Welcome! I can help you find similar movies based on a title.
Type 'quit' or 'exit' to end the conversation.

Enter a movie title: interstellar

Found movie: Interstellar

Here are similar movies you might enjoy:
1. Gattaca
2. Space Pirate Captain Harlock
3. Space Cowboys
4. Starship Troopers
5. Final Destination 2

Enter a movie title: Penitentiary

Found movie: Penitentiary

Here are similar movies you might enjoy:
1. The Texas Chainsaw Massacre: The Beginning
2. Wild Hogs
3. Child's Play 2
4. Crazy/Beautiful
5. Wicked Blood

Enter a movie title: avatar

Found movie: Avatar

Here are similar movies you might enjoy:
1. The Matrix
2. The Inhabited Island
3. Apollo 18
4. Tears of the Sun
5. Green Zone

Enter a movie title: minions

Found movie: Minions

Here are similar movies you might enjoy:
1. Despicable Me 2
2. Despicable Me
3. Teenage Mutant Ninja Turtles: Out of the Shadows
4. Superman
5. Rise of the Guardians

Enter a movie title: despicable me

Found